In [8]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

template = """

    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You:
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)



In [6]:
invoke_chain("I love in Seoul")

content="That's great! Seoul is a vibrant and bustling city known for its rich history, modern architecture, and delicious cuisine. It's also home to many cultural landmarks and attractions like Gyeongbokgung Palace, N Seoul Tower, and the vibrant shopping district of Myeongdong. If you have any specific questions or need recommendations about Seoul, feel free to ask!"


In [9]:
invoke_chain("What is my name?")

content="I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. My primary function is to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I can assist you with?"


In [8]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is Nico'),
  AIMessage(content='Hello Nico! How can I assist you today?'),
  HumanMessage(content='I love in Seoul'),
  AIMessage(content="That's great, Nico! Seoul is a vibrant and exciting city. Is there anything specific you would like to know or discuss about Seoul? I'm here to help!"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Nico.')]}